Thanks to https://github.com/hughesj919/HomographyEstimation

For running this, we need opencv module with SIFT features.  You can install it in Anaconda using the command

conda install -c menpo opencv
pip install opencv-contrib-python==3.4.2.16
pip install --user ipykernel
python -m ipykernel install --user --name=py35_dummy

In [ ]:
import sys
sys.executable

In [ ]:
import cv2
import numpy as np
import getopt
import sys
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
def findFeatures(img):
    print("Finding Features...")
    sift = cv2.xfeatures2d.SIFT_create() #cv2.SIFT()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    img_with_feature = np.copy(img)
    img_with_feature = cv2.drawKeypoints(img, keypoints, img_with_feature)
    #cv2.imwrite('sift_keypoints.png', img)

    return keypoints, descriptors, img_with_feature

In [ ]:
def matchFeatures(kp1, kp2, desc1, desc2, img1, img2):
    print("Matching Features...")
    matcher = cv2.BFMatcher(cv2.NORM_L2, True)
    matches = matcher.match(desc1, desc2)
    matchImg = drawMatches(img1,kp1,img2,kp2,matches)
    return matches, matchImg

In [ ]:
def calculateHomography(correspondences):
    #loop through correspondences and create assemble matrix
    aList = []
    for corr in correspondences:
        p1 = np.matrix([corr.item(0), corr.item(1), 1])
        p2 = np.matrix([corr.item(2), corr.item(3), 1])

        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),
              p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,
              p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        aList.append(a1)
        aList.append(a2)

    matrixA = np.matrix(aList)

    #svd composition
    u, s, v = np.linalg.svd(matrixA)

    #reshape the min singular value into a 3 by 3 matrix
    h = np.reshape(v[8], (3, 3))

    #normalize and now we have h
    h = (1/h.item(8)) * h
    return h


#
#Calculate the geometric distance between estimated points and original points
#
def geometricDistance(correspondence, h):

    p1 = np.transpose(np.matrix([correspondence[0].item(0), correspondence[0].item(1), 1]))
    estimatep2 = np.dot(h, p1)
    estimatep2 = (1/estimatep2.item(2))*estimatep2

    p2 = np.transpose(np.matrix([correspondence[0].item(2), correspondence[0].item(3), 1]))
    error = p2 - estimatep2
    return np.linalg.norm(error)


#
#Runs through ransac algorithm, creating homographies from random correspondences
#
def ransac(corr, thresh):
    maxInliers = []
    finalH = None
    for j in range(200):
        #find 4 random points to calculate a homography
        corr1 = corr[random.randrange(0, len(corr))]
        corr2 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((corr1, corr2))
        corr3 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr3))
        corr4 = corr[random.randrange(0, len(corr))]
        randomFour = np.vstack((randomFour, corr4))

        #call the homography function on those points
        h = calculateHomography(randomFour)
        inliers = []

        for i in range(len(corr)):
            d = geometricDistance(corr[i], h)
            if d < 5:
                inliers.append(corr[i])

        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
        print ("In iteration : " + str(j) + " Corr size: " + str(len(corr)) + " NumInliers: " + str(len(inliers)) + " Max inliers: " + str(len(maxInliers)))

        if len(maxInliers) > (len(corr)*thresh):
            break
    return finalH, maxInliers

In [ ]:
# This draws matches and optionally a set of inliers in a different color
# Note: I lifted this drawing portion from stackoverflow and adjusted it to my needs because OpenCV 2.4.11 does not
# include the drawMatches function
def drawMatches(img1, kp1, img2, kp2, matches, inliers = None):
    # Create a new output image that concatenates the two images together
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image to the left
    out[:rows1,:cols1,:] = np.dstack([img1, img1, img1])

    # Place the next image to the right of it
    out[:rows2,cols1:cols1+cols2,:] = np.dstack([img2, img2, img2])

    # For each pair of points we have between both images
    # draw circles, then connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx

        # x - columns, y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt

        inlier = False

        if inliers is not None:
            for i in inliers:
                if i.item(0) == x1 and i.item(1) == y1 and i.item(2) == x2 and i.item(3) == y2:
                    inlier = True

        # Draw a small circle at both co-ordinates
        cv2.circle(out, (int(x1),int(y1)), 4, (255, 0, 0), 1)
        cv2.circle(out, (int(x2)+cols1,int(y2)), 4, (255, 0, 0), 1)

        # Draw a line in between the two points, draw inliers if we have them
        if inliers is not None and inlier:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (0, 255, 0), 1)
        elif inliers is not None:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (0, 0, 255), 1)

        if inliers is None:
            cv2.line(out, (int(x1),int(y1)), (int(x2)+cols1,int(y2)), (255, 0, 0), 1)

    return out

In [ ]:
inlier_ratio = .6
img_name1 = 'Shakuntala1.jpg'
img_name2 = 'Shakuntala2.jpg'
img1 = cv2.imread(img_name1)
img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2RGB)
img2 = cv2.imread(img_name2)
img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
f.tight_layout()
ax1.imshow(img1)
ax1.set_title("Image 1", fontsize=50)
ax2.imshow(img2)
ax2.set_title("Image 2", fontsize=50)

In [ ]:
correspondenceList = []


kp1, desc1, img1_with_feature = findFeatures(img1_gray)
kp2, desc2, img2_with_feature = findFeatures(img2_gray)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
f.tight_layout()
ax1.imshow(img1_with_feature)
ax1.set_title(str(len(kp1)) + " features", fontsize=30)
ax2.imshow(img2_with_feature)
ax2.set_title(str(len(kp2)) + " features", fontsize=30)

In [ ]:
keypoints = [kp1,kp2]
matches, matchImg = matchFeatures(kp1, kp2, desc1, desc2, img1_gray, img2_gray)
for match in matches:
    (x1, y1) = keypoints[0][match.queryIdx].pt
    (x2, y2) = keypoints[1][match.trainIdx].pt
    correspondenceList.append([x1, y1, x2, y2])

corrs = np.matrix(correspondenceList)

#run ransac algorithm
finalH, inliers = ransac(corrs, inlier_ratio)
#print("Final homography: " + str(finalH))
#print("Final inliers count: " + str(len(inliers))

matchImg_inlier = drawMatches(img1_gray,kp1,img2_gray,kp2,matches,inliers)


In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(20, 9))
f.tight_layout()
ax1.imshow(matchImg)
ax1.set_title(str(len(correspondenceList)) + " matches", fontsize=30)

In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(20, 9))
f.tight_layout()
ax1.imshow(matchImg_inlier)
ax1.set_title(str(len(inliers)) + " inliers", fontsize=30)

In [ ]:
left_im_point = [150, 200]
proj_left = np.array(left_im_point + [1]).reshape((3,1))
proj_right = finalH*proj_left
right_im_point = [proj_right[0,0]/proj_right[2,0], proj_right[1,0]/proj_right[2,0]]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))

f.tight_layout()
ax1.imshow(img1)
ax1.set_title("Image 1", fontsize=30)

patches_plot_left = [patches.Circle((left_im_point[0], left_im_point[1]), radius=2, color='green')]
for p in patches_plot_left:
    ax1.add_patch(p)


ax2.imshow(img2)
ax2.set_title("Image 2", fontsize=30)
patches_plot_right = [patches.Circle((right_im_point[0], right_im_point[1]), radius=2, color='green')]
for p in patches_plot_right:
    ax2.add_patch(p)